In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import os, glob

# Create Data Structures

In [2]:
N_IMAGES_TRAIN = 20_000
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 100

In [3]:
#First we define a custom dataset class to be able to create the DataLoader properly

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, labels_file, img_dir, separator=',', img_type="png",transform=None, max_images=None):
        # As labels, we select the 'c' column of the dataset, which contains the number
        # of circles of the image
        aux = pd.read_csv(labels_file,sep=separator)
        aux.drop(columns=aux.columns[0], axis=1,inplace=True)
        aux.drop(columns=aux.columns[1:], axis=1, inplace=True)
        self.img_labels = aux.to_numpy().squeeze()
        if max_images is not None:
            self.img_labels = self.img_labels[:max_images]
        self.img_dir = img_dir
        self.transform = transform
        self.img_type = img_type

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        # To get the images, we transform the index to a 5 long string (so 28 -> 00028)
        # and we use that to read the image.
        img_name = str(idx).zfill(5)+'.'+self.img_type
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)
        label = self.img_labels[idx,]
        if self.transform:
            image = self.transform(image)
            image = image.float()
        label = torch.tensor(label, dtype=torch.float32)
        return (image, label)

In [4]:
#Testing custom dataset is correctly constructed
#dummy_dataset = CustomImageDataset(labels_file='data/val/dades.csv', img_dir='data/val', separator=';')
#batch = next(iter(dummy_dataset))
#plt.imshow(batch[0], cmap='gray')
#plt.show()
#print(batch[1])

In [5]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128))
])


train_dataset = CustomImageDataset(labels_file='data/train/dades.csv',img_dir='data/train', separator=';',max_images=20_000
                                  ,transform=transform)
test_dataset = CustomImageDataset(labels_file='data/val/dades.csv',img_dir='data/val',separator=';',transform=transform)

train_loader = DataLoader(train_dataset, TRAIN_BATCH_SIZE)
test_loader = DataLoader(test_dataset, TEST_BATCH_SIZE)

In [6]:
class FullyConvNet(nn.Module):
    def __init__(self):
        super(FullyConvNet, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding="same")
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding="same")
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding="same")
        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size = 3, stride=1, padding="same")
        self.linear1 = nn.Linear(8*8*128,256)
        self.linear2 = nn.Linear(256,1)
        self.relu =nn.ReLU()

    def forward(self, x):
        x = self.conv_1(x)
        x = self.max_pool(x) #out = 64x64
        x = self.conv_2(x)
        x = self.max_pool(x) #out = 32x32
        x = self.conv_3(x)
        x = self.max_pool(x) # out = 16x16
        x = self.conv_4(x)
        x = self.max_pool(x) # out = 8x8 * 128
        x = torch.flatten(x,1)
        x = self.linear1(x)
        x = self.relu(x)
        output = self.linear2(x)
        
        return output.squeeze()

In [7]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        output = model(data)

        loss = loss_func(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item()/ len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_func(output, target)
            output = torch.round(output)
            correct += output.eq(target.view_as(output)).sum().item()
 
  
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [8]:
use_cuda = True
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = 20
lr = 0.001

model = FullyConvNet().to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # !!!

print("Parameters ",pytorch_total_params)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = torch.nn.MSELoss()

# Guardam el valor de peèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)

Parameters  2194817
Train Epoch: 0 [0/20000 (0%)]	Loss: 1.680200, Average: 0.013127
Train Epoch: 0 [12800/20000 (64%)]	Loss: 0.499701, Average: 0.003904

Train set: Average loss: 0.0041


Test set: Average loss: 0.0039, Accuracy: 1055/2000 (53%)

Train Epoch: 1 [0/20000 (0%)]	Loss: 0.349566, Average: 0.002731
Train Epoch: 1 [12800/20000 (64%)]	Loss: 0.373633, Average: 0.002919

Train set: Average loss: 0.0028


Test set: Average loss: 0.0045, Accuracy: 1030/2000 (52%)

Train Epoch: 2 [0/20000 (0%)]	Loss: 0.372009, Average: 0.002906
Train Epoch: 2 [12800/20000 (64%)]	Loss: 0.205607, Average: 0.001606

Train set: Average loss: 0.0014


Test set: Average loss: 0.0018, Accuracy: 1495/2000 (75%)

Train Epoch: 3 [0/20000 (0%)]	Loss: 0.162391, Average: 0.001269
Train Epoch: 3 [12800/20000 (64%)]	Loss: 0.054401, Average: 0.000425

Train set: Average loss: 0.0005


Test set: Average loss: 0.0006, Accuracy: 1876/2000 (94%)

Train Epoch: 4 [0/20000 (0%)]	Loss: 0.056625, Average: 0.000442
Train Ep

KeyboardInterrupt: 